In [2]:
try:
  import mne
except:
  ! pip install mne
  import mne

import os
import torch
from torch import nn
import pandas as pd
import torchaudio
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from mne.datasets.sleep_physionet.age import fetch_data
from google.colab import drive
import numpy as np

In [3]:
# ONLY IF YOU'RE USING GOOGLE COLAB
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# write your path for the dataset
path='/content/drive/MyDrive/Lessons/Machine Learning/Project/datasets'
#fetch_data(subjects=range(83), on_missing='warn',path=path)

In [5]:
path=path+'/physionet-sleep-data/'

In [6]:
files_paths=os.listdir(path) #gets path for PSG and Hypnogram files
psgs_paths=[]
hyps_paths=[]

for file_name in files_paths:
  if 'PSG.edf' in file_name:
    psgs_paths.append(path+ f'/{file_name}')
  elif 'Hypnogram.edf' in file_name:
    hyps_paths.append(path+ f'/{file_name}')

In [ ]:
# reads PSG and Hypnogram files
psgs_sigs=list(map(mne.io.read_raw_edf, psgs_paths))
hyps_labels=list(map(mne.read_annotations, hyps_paths))
hyps_labels[0]

In [ ]:
for i,sig in enumerate(psgs_sigs):
  sig=sig.drop_channels(sig.ch_names[1:])
  sig.set_annotations(hyps_labels[i])

  if '?' in hyps_labels[i][-1]['description']: # removes the parts of signal with "?" label
    last_wake=-2
  else:
    last_wake=-1

  # keeps only 30 mins after waking up
  if hyps_labels[i][last_wake]['duration']<=30*60:
    tmax=hyps_labels[i][last_wake]['onset']+hyps_labels[i][last_wake]['duration']-1
  else:
    tmax=hyps_labels[i][last_wake]['onset']+30*60

  # keeps only 30 mins before sleep
  if  hyps_labels[i][0]['duration']-30*60<=0:
    tmin=hyps_labels[i][0]['duration']
  else:
    tmin=hyps_labels[i][0]['duration']-30*60
  sig.crop(tmin=tmin, tmax=tmax)


In [9]:
annotation_map_id={
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4}


for i,sig in enumerate(psgs_sigs):
  events, event_id=mne.events_from_annotations(sig, event_id=annotation_map_id, chunk_duration=30.0)
  sig_epochs=mne.Epochs(sig, events=events, tmin=0, tmax=29.99 # no overlap
                      , baseline=(0,0), preload=True)

  data=[]
  labels=[]
  sig_epoched_annotations=sig_epochs.get_annotations_per_epoch()
  for j,l in enumerate(sig_epoched_annotations):
    labels.append(annotation_map_id[l[0][2]])
    data.append(sig_epochs[j])

  data = np.stack(data,0)
  labels=np.stack(labels,0)

  if i==0:
    all_sigs_datas=data
    all_sigs_labels=labels
  else:
    all_sigs_datas=np.append(all_sigs_datas,data,axis=0)
    all_sigs_labels=np.append(all_sigs_labels,labels,axis=0)

  if i==1:
    break

all_sigs_datas=np.squeeze(all_sigs_datas,axis=1)
all_sigs_labels=np.array(pd.get_dummies(all_sigs_labels))

Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1103 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 1103 events and 3000 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1186 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 1186 events and 3000 original time points ...
0 bad epochs dropped


In [10]:
class EEGDataset(Dataset):
  def __init__(self,all_sigs_datas,all_sigs_labels):
    all_sigs_labels=np.expand_dims(all_sigs_labels,axis=1)
    self.all_sigs_datas=torch.from_numpy(all_sigs_datas)
    self.all_sigs_labels=torch.from_numpy(all_sigs_labels)

  def __len__(self):
    return len(self.all_sigs_labels)

  def __getitem__(self, index):
    #label=torch.from_numpy(self.all_sigs_labels[index])
    #sig=torch.from_numpy(self.all_sigs_datas[index])
    label=self.all_sigs_labels[index]
    sig=self.all_sigs_datas[index]
    return sig, label


#----------------------------
eeg_dataset=EEGDataset(all_sigs_datas,all_sigs_labels)
loaders = {
    'train' : torch.utils.data.DataLoader(eeg_dataset,
                                          batch_size=128,
                                          shuffle=True),

    'test'  : 0
}

##################################################################

In [207]:
class RSEBlock(nn.Module):
    def __init__(self, channel=128, inplanes=128, planes=30, stride=1, groups=1,
                 base_width=64, dilation=1, norm_layer=None,
                 *, reduction=16):
        super(RSEBlock, self).__init__()
        expansion = 1

        self.avg_pool = nn.AdaptiveAvgPool1d(1)

        self.fc = nn.Sequential( # its the auto encoder
            nn.Linear(30, 30 // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(30 // reduction, 30, bias=False),
            nn.Sigmoid()
        )

        self.conv1 = nn.Conv1d(in_channels=128, out_channels=30, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(planes, planes, 1)
        self.bn2 = nn.BatchNorm1d(planes)

    def forward(self, x):
        residual = x
        out = self.conv1(residual)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        b, c, _ = out.size()
        y = self.avg_pool(out).view(b, c)
        y = self.fc(y).view(b, c, 1)
        y=out * y.expand_as(out)
        y += out
        y = self.relu(y)

        return y

class MultiResCNN(nn.Module):
    def __init__(self):
        super(MultiResCNN, self).__init__()
        drate = 0.5
        self.GELU = nn.GELU()  # for older versions of PyTorch.  For new versions use nn.GELU() instead.
        self.cnn1 = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=50, stride=6, bias=False, padding=24),
            nn.BatchNorm1d(64),
            self.GELU,
            nn.MaxPool1d(kernel_size=8, stride=2, padding=4),
            nn.Dropout(0.5),

            nn.Conv1d(64, 128, kernel_size=8, stride=1, bias=False, padding=4),
            nn.BatchNorm1d(128),
            self.GELU,

            nn.Conv1d(128, 128, kernel_size=8, stride=1, bias=False, padding=4),
            nn.BatchNorm1d(128),
            self.GELU,

            nn.MaxPool1d(kernel_size=4, stride=4, padding=2)
        )

        self.cnn2 = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=400, stride=50, bias=False, padding=200),
            nn.BatchNorm1d(64),
            self.GELU,
            nn.MaxPool1d(kernel_size=4, stride=2, padding=2),
            nn.Dropout(0.5),

            nn.Conv1d(64, 128, kernel_size=7, stride=1, bias=False, padding=3),
            nn.BatchNorm1d(128),
            self.GELU,

            nn.Conv1d(128, 128, kernel_size=7, stride=1, bias=False, padding=3),
            nn.BatchNorm1d(128),
            self.GELU,
            nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        )
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x1 = self.cnn1(x)
        x2 = self.cnn2(x)
        x_concat = torch.cat((x1, x2), dim=2)
        x_concat = self.dropout(x_concat)
        return x_concat

class PSGClassifier(nn.Module):
    def __init__(self):
        super(PSGClassifier, self).__init__()
        d_model = 80
        afr_reduced_cnn_size = 30

        self.mrcnn = MultiResCNN()
        self.rse= RSEBlock()
        self.flatten=nn.Flatten()
        self.fc = nn.Linear(d_model * afr_reduced_cnn_size, 5)

    def forward(self, x):
        x_feat = self.mrcnn(x)
        x_feat=self.rse(x_feat)
        x_feat=self.flatten(x_feat)
        final_output = self.fc(x_feat)
        return final_output

In [204]:
# Setting the device
#device = "cuda" if torch.cuda.is_available() else "cpu"
device="cpu"
print(device)

cpu


In [ ]:
import torch
#dummy=torch.randn(size=(2,1,3000)).to(device)

psg_classifier=PSGClassifier().to(device)
dummy, _ = next(iter(loaders['train']))
dummy=dummy.to(device)
dummy=dummy.float()
answer=psg_classifier(dummy)
print(answer)

In [211]:
optimizer=torch.optim.Adam(psg_classifier.parameters(),lr=0.01,weight_decay=0.01,betas=(0.9, 0.999), eps=1e-08,amsgrad=True)
loss_fn= nn.MSELoss()

num_epochs=30

def train_step(num_epochs, psg_classifier, loaders):
  psg_classifier.train()

  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loaders["train"]):

      images=images.type(torch.FloatTensor)
      labels=labels.type(torch.FloatTensor)
      images,labels= images.to(device), labels.to(device)

      output=psg_classifier(images)
      loss=loss_fn(output, labels)
      #print(f"Epoch: {epoch} | Loss: {loss.item():.4f}")
      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

    print(f"Epoch: {epoch} | Loss: {loss.item():.4f}")

In [ ]:
train_step(num_epochs, psg_classifier, loaders)

In [ ]:
psg_classifier(dummy)